In [48]:
import torch
from models import skipgram
import pandas as pd
import wandb
import os

run = wandb.init(project="word2vec")
artifact = run.use_artifact("model-weights:latest")
datadir = artifact.download()

wandb:   1 of 1 files downloaded.  


In [ ]:
# vocab = pd.read_csv(f"data/vocab{'__mini' if skipgram.MINIMODE else ''}.generated.csv", index_col='token')
vocab = pd.read_csv(f"data/vocab.generated.csv", index_col='token')

model = skipgram.Model(vocab.size + 1, skipgram.EMBEDDING_DIM)

# state_dict = torch.load('./data/weights.generated.pt')
state_dict = torch.load(os.path.join(datadir, 'weights.generated.pt'), map_location=torch.device('cpu'))

def getIdFromToken(token: str):
    try:
        return int(vocab.at[token, 'id'])
    except:
        return int(vocab.at['unk', 'id'])

#find the n most similar words
def most_similar(word, n=5):
    word_idx = getIdFromToken(word)
    A =  state_dict['embeddings.weight'][word_idx].unsqueeze(0)
    word_similarities = []
    for token, ids in vocab.iterrows():
        id = ids.values[0]
        B =  model.embeddings.weight[id].unsqueeze(0)
        cosine_similarity = torch.nn.functional.cosine_similarity(A, B, dim=1)
        word_similarities.append((token, cosine_similarity.item()))
    word_similarities = sorted(word_similarities, key=lambda x: x[1], reverse=True)
    return word_similarities[:n]

girl_id = getIdFromToken('girl')
girl = model.embeddings.weight[girl_id]
boy_id = getIdFromToken('boy')
boy = model.embeddings.weight[boy_id]
torch.nn.functional.cosine_similarity(girl, boy, dim=1)
# most_similar('python')

1930


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)